In [1]:
import pandas as pd
import numpy as np
import importlib.machinery
es = importlib.machinery.SourceFileLoader('extrasense','/home/sac086/extrasensory/extrasense/extrasense.py').load_module()
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit, ShuffleSplit, KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler

# Testing a Personal Model
This is just so that I can get a sense of whether the data is working out.

In [ ]:
def test_personal_model(user_id):
    kf = KFold(n_splits=3)

    user_df = es.get_data_from_user_id(user_id, data_type="activity", labeled_only=True)
    timestamps_series = user_df.pop('timestamp')
    scores = []
    for training_ind, test_ind in kf.split(user_df):
        train_df = user_df.iloc[training_ind]
        test_df = user_df.iloc[test_ind]

        train_labels = train_df.pop("label")
        test_labels = test_df.pop("label")
        

        clf = RandomForestClassifier()

        # standard scale training
        scaler = StandardScaler().fit(train_df)
        scaled_train_df = scaler.transform(train_df)
        scaled_test_df = scaler.transform(test_df)

        clf.fit(scaled_train_df, train_labels)

        # standard scale testing
        predictions = clf.predict(scaled_test_df)
        score = accuracy_score(test_labels, predictions)
        scores.append(score)
    return np.mean(scores)

In [ ]:
user_scores = {}
for user_id in es.user_ids:
    mean_score = test_personal_model(user_id)
    user_scores[user_id] = mean_score

In [ ]:
scores = [v for v in user_scores.values()]

In [ ]:
np.mean(scores)

# Testing an impersonal model
Leave-One Participant out design

Sampling personal test data with ShuffleSplit method which will randomly select personal data
making no gaurentee that the distribution of test data is similar to the distribution of training data

In [2]:
user_df = es.get_data_from_user_id(es.user_ids[0], data_type="activity", labeled_only=True)

/home/sac086/extrasensory/extrasense/extrasense.py:159: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  user_df = user_df[null_mask]


In [14]:
impersonal_df = es.get_impersonal_data(es.user_ids[0], data_type="activity", labeled_only=True)

In [15]:
impersonal_df = pd.concat(impersonal_df, ignore_index=True)

In [16]:
features_df = impersonal_df[impersonal_df.columns.intersection(es.acc_col_name)]

In [17]:
labels_df = es.clean_labels(impersonal_df[impersonal_df.columns.intersection(es.acc_labels_name)])

In [18]:
impersonal_df = pd.concat((features_df,labels_df,impersonal_df['timestamp']), axis=1)

In [19]:
impersonal_df.head()

,raw_acc:magnitude_stats:mean,raw_acc:magnitude_stats:std,raw_acc:magnitude_stats:moment3,raw_acc:magnitude_stats:moment4,raw_acc:magnitude_stats:percentile25,raw_acc:magnitude_stats:percentile50,raw_acc:magnitude_stats:percentile75,raw_acc:magnitude_stats:value_entropy,raw_acc:magnitude_stats:time_entropy,raw_acc:magnitude_spectrum:log_energy_band0,...,raw_acc:3d:mean_y,raw_acc:3d:mean_z,raw_acc:3d:std_x,raw_acc:3d:std_y,raw_acc:3d:std_z,raw_acc:3d:ro_xy,raw_acc:3d:ro_xz,raw_acc:3d:ro_yz,0,timestamp
0,1.000371,0.007671,-0.016173,0.027860,0.998221,1.000739,1.003265,0.891038,6.684582,5.045157,...,0.000406,-1.000342,0.004052,0.003310,0.007697,-0.329743,0.382219,-0.121107,NaN,1449601597
1,1.000243,0.003782,-0.002713,0.007046,0.998463,1.000373,1.002088,1.647929,6.684605,5.044516,...,0.000355,-1.000211,0.002911,0.002960,0.003792,0.202860,0.335481,0.105470,NaN,1449601657
2,1.000811,0.002082,-0.001922,0.003575,0.999653,1.000928,1.002032,1.960286,6.684610,5.043892,...,-0.000170,-1.000802,0.002262,0.001760,0.002086,0.111225,0.488020,0.154312,NaN,1449601717
3,1.001245,0.004715,-0.002895,0.008881,0.999188,1.001425,1.003500,1.614524,6.684601,5.043505,...,0.002558,-1.001208,0.003094,0.003091,0.004724,-0.107052,0.351430,-0.054253,NaN,1449601777
4,1.001354,0.065186,-0.096520,0.165298,1.000807,1.002259,1.003631,0.837790,6.682252,5.045126,...,0.024336,-0.996488,0.054192,0.067662,0.074172,-0.106908,0.460092,-0.021773,SITTING,1449601855


In [ ]:
type(impersonal_df)

In [ ]:
list(impersonal_df.columns)

In [ ]:
def test_impersonal_model(user_id):
    personal_df = es.get_data_from_user_id(user_id, data_type="activity", labeled_only=True)
    impersonal_df
    timestamps_series = user_df.pop('timestamp')
    scores = []

    for training_ind, test_ind in kf.split(user_df):
        train_df = user_df.iloc[training_ind]
        test_df = user_df.iloc[test_ind]

        train_labels = train_df.pop("label")
        test_labels = test_df.pop("label")
        

        clf = RandomForestClassifier()

        # standard scale training
        scaler = StandardScaler().fit(train_df)
        scaled_train_df = scaler.transform(train_df)
        scaled_test_df = scaler.transform(test_df)

        clf.fit(scaled_train_df, train_labels)

        # standard scale testing
        predictions = clf.predict(scaled_test_df)
        score = accuracy_score(test_labels, predictions)
        scores.append(score)
    return np.mean(scores)